In [ ]:
!pip uninstall opencv-python -y
!pip install "opencv-python <= 4.5.5"

Found existing installation: opencv-python 4.1.2.30
Uninstalling opencv-python-4.1.2.30:
  Successfully uninstalled opencv-python-4.1.2.30
     |████████████████████████████████| 60.3 MB 1.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
# 내 구글 드라이브에 연동
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# 필요한 패키지와 모듈을 불러옴
import cv2
import numpy as np
import time
import io
import base64
from IPython.display import HTML

In [ ]:
# OpenCV 버전 확인
print(cv2.__version__)

4.5.4


In [ ]:
# Detection 하기 전에 원본 동영상을 Display
video = io.open('/content/gdrive/MyDrive/Custom_YOLO4/custom_fruit.mp4', 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video width="30%" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4"/>
             </video>'''.format(encoded.decode('ascii')))

Output hidden; open in https://colab.research.google.com to view.

In [6]:
file_name = '/content/gdrive/MyDrive/Custom_YOLO4/custom_fruit.mp4'  # Detection 할 원본 동영상
min_confidence = 0.5  # detection 으로 인정할 최소 확률(신뢰도) 지정
nms_threshold = 0.4   # non-max suppression threshold
output_name = 'fruit_output_video.mp4'  # Detection 된 output 동영상 
elapsed_time = 0       # 총 경과시간 초기화 

In [7]:
def detectAndDisplay(frame):
    start_time = time.time()

    global min_confidence
    global nms_threshold
    classIds, scores, boxes = model.detect(frame, confThreshold=min_confidence, nmsThreshold=nms_threshold)

    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        x, y, w, h = boxes[i]
        label = '{} {:,.2%}'.format(str(classes[classIds[i]]), scores[i])
        print(i, label)
        color = color_lists[classIds[i]]
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 1)
        cv2.rectangle(frame, (x, y-25), (x + w, y), color, -1)
        cv2.putText(frame, label, (x+2, y - 10), font, 1, (255, 255, 255), 1)

    process_time = time.time() - start_time
    global elapsed_time
    elapsed_time += process_time   # 총 경과시간 누적
    print("=== A frame took {:.3f} seconds".format(process_time))

    # video 를 disk 에 output 하기 위해 writer 를 초기화한다.
    global writer
    if writer is None and output_name is not None:
        fourcc = cv2.VideoWriter_fourcc(*"DIVX")
        writer = cv2.VideoWriter(output_name, fourcc, 20,
                (frame.shape[1], frame.shape[0]), True)
        
    # disk 에 frame 을 write 합니다.
    if writer is not None:
        writer.write(frame)

In [8]:
# Load YOLO
net = cv2.dnn.readNetFromDarknet("/content/gdrive/MyDrive/Custom_YOLO4/custom/custom-test-yolo4.cfg", "/content/gdrive/MyDrive/Custom_YOLO4/backup/custom-train-yolo4_final.weights")
classes = []   # detection 할 Object(Class) list 배열을 정의
with open("/content/gdrive/MyDrive/Custom_YOLO4/custom/classes.names", "r") as f:
   classes = [line.strip() for line in f.readlines()]   # 3개의 Object(class)를 구분할 수 있는 Object의 이름을 classes 배열에 넣어준다.
color_lists = np.random.uniform(0, 255, size=(len(classes), 3))  # Object 마다 컬러를 하나씩 다르게 지정

model = cv2.dnn_DetectionModel(net)
model.setInputParams(scale=1./255, size=(416, 416), swapRB=True, crop=False)

# 원본 동영상에서 video stream을 읽어온다.
cap = cv2.VideoCapture(file_name)
writer = None
if not cap.isOpened:
    print('--(!)Error opening video capture')
    exit(0)
while True:
    ret, frame = cap.read()
    if frame is None:
        # close the video file pointers
        cap.release()
        # close the writer point
        writer.release()
        print('--(!) No captured frame -- Break!')
        print("elapsed time {:.3f} seconds".format(elapsed_time))
        break
    detectAndDisplay(frame)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
1 Tomato 99.90%
2 Tomato 94.09%
3 Apple 99.94%
4 Apple 99.89%
=== A frame took 1.607 seconds
0 Oriental melon 99.58%
1 Tomato 99.88%
2 Tomato 86.15%
3 Tomato 76.17%
4 Apple 99.94%
5 Apple 99.89%
=== A frame took 1.629 seconds
0 Oriental melon 99.43%
1 Tomato 99.88%
2 Tomato 96.70%
3 Tomato 72.37%
4 Apple 99.94%
5 Apple 99.88%
=== A frame took 1.603 seconds
0 Oriental melon 99.61%
1 Tomato 99.87%
2 Tomato 99.72%
3 Tomato 85.24%
4 Apple 99.94%
5 Apple 99.80%
=== A frame took 1.581 seconds
0 Oriental melon 99.54%
1 Tomato 99.87%
2 Tomato 99.86%
3 Tomato 93.18%
4 Apple 99.93%
5 Apple 99.83%
=== A frame took 1.642 seconds
0 Oriental melon 99.47%
1 Tomato 99.91%
2 Tomato 99.86%
3 Tomato 96.61%
4 Apple 99.93%
5 Apple 99.85%
=== A frame took 1.617 seconds
0 Oriental melon 99.55%
1 Tomato 99.88%
2 Tomato 99.83%
3 Apple 99.93%
4 Apple 99.86%
5 Apple 86.81%
=== A frame took 1.630 seconds
0 Oriental melon 99.59%
1 Tomato 99.87%
2 Tomato 98.37%
3 Apple 99.93%
4 A